# Birthwt Dataset_Multiple Linear Regression

url : http://rstudio-pubs-static.s3.amazonaws.com/189354_277dfb3a83a34a2abaae855b90fcf269.html

## [Multiple Linear Regression 다중회귀]

다중회귀는 단순회귀와 다르게 예측변수가 2개 이상일 때 사용한다. 예측변수가 많을 경우 분석에 앞서 모형선택 과정을 거친다.예측변수의 선택은 다음과 같다. 예측변수끼리 상관관계가 높다는 것은 중복된 정보가 반영된다는 의미이다. 변수 낭비뿐만 아니라 계산상으로도 문제를 일으켜 예측변수를 의미없게 만들수 있다. (다중공선성 Multicolinearity)

- 반응변수와 높은 상관관계
- 선택된 예측변수끼리 낮은 상관관계
- 소수정예의 원칙

### [ EA Exploratory Analysis 모형선택]

변수들간에 높은 상관관계가 있을 경우 전진선택법, 너무 많지 않은 경우 후진제거법을 권장한다.

1. [All Possible Regression 전체선택법] : 가증한 모든 예측변수드의 조합으로 모형을 만듬.
1. [Forword Selection 전진선택법] : 가장 유의한 변수 순으로 하나씩 추가.
1. [Backword Elimination 후진제거법] : 모든 변수를 넣고 기여도 낮은 변수를 하나씩 제거.
1. [Stepwise Method 단계별방법] : 전진선택법으로 변수를 추가후 중요도에 따라 추가 또는 제거.


----

## 1. birthwt Dataset

MASS 패키지에 포함되어 있는 birthwt는 1986년 미국 Messachysetts주 Springfield에 있는 Babystate Medical Center에서 수집한 자료로 189명분의 나이, 인종 등의 자료이다.

1. Age : 엄마의 나이
1. LTW(Last Menstrual Weight) : 마지막 생리기간의 엄마의 몸무게
1. Race : 인종
1. Smoke : 임신기간중 흡연 여부
1. PTL(PreTerm Labor)  : 과거 조산 횟수
1. HT(HyperTension) : 고혈압의 기왕력
1. UI(Urerine Irrirabillity) : 자궁과민성 여부
1. FTV(First Trimester Visits) : 첫 임신 3개월동안 의사 방문 횟수
1. BWT(Birth Weight in Grams) : 출생시 몸무게(g)

In [3]:
require(MASS)
tail(birthwt)

,low,age,lwt,race,smoke,ptl,ht,ui,ftv,bwt
78,1,14,101,3,1,1,0,0,0,2466
79,1,28,95,1,1,0,0,0,2,2466
81,1,14,100,3,0,0,0,0,2,2495
82,1,23,94,3,1,0,0,0,0,2495
83,1,17,142,2,0,0,1,0,0,2495
84,1,21,130,1,1,0,1,0,3,2495


In [4]:
str(birthwt)

'data.frame':	189 obs. of  10 variables:
 $ low  : int  0 0 0 0 0 0 0 0 0 0 ...
 $ age  : int  19 33 20 21 18 21 22 17 29 26 ...
 $ lwt  : int  182 155 105 108 107 124 118 103 123 113 ...
 $ race : int  2 3 1 1 1 3 1 3 1 1 ...
 $ smoke: int  0 0 1 1 1 0 0 0 1 1 ...
 $ ptl  : int  0 0 0 0 0 0 0 0 0 0 ...
 $ ht   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ ui   : int  1 0 0 1 1 0 0 0 0 0 ...
 $ ftv  : int  0 3 1 2 0 0 1 1 1 0 ...
 $ bwt  : int  2523 2551 2557 2594 2600 2622 2637 2637 2663 2665 ...


불러온 데이터의 칼럼 정보를 확인한다.
- 전부 숫자로 이루어져 있고 총 189개의 정보
- 총 10개의 변수(=칼럼)을 가짐

##### Notice
**[Race 인종]**변수는 엄마가 백인일 경우 1, 흑인일 경우 2, 그 외일 경우 3으로 되어 있는 [Categorycal 범주형] 데이터이다. 그냥 race를 사용할 경우 [Continuous연속형] 범주로 인식하기 때문에 fector로 사용해야 한다.

## 2. Linear Model

bwt를 반응변수로 agem lwt, race, smoke, ptl, ht, ui를 독립변수로 하여 다중회귀분석을 해본다.
- **anova()** : 분산을 테이블 형식으로 계산하는 기능. **분산분석표**를 만듬

In [5]:
out <- lm(bwt~ age+lwt+factor(race)+smoke+ptl+ht+ui, data=birthwt)
anova(out)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
age,1,815483.2,815483.2,1.938012,0.1656021823
lwt,1,2967338.6,2967338.6,7.051937,0.0086283664
factor(race),2,4750631.8,2375315.9,5.644984,0.0041901349
smoke,1,6291917.5,6291917.5,14.952863,0.0001538420
ptl,1,732500.9,732500.9,1.740802,0.1887129935
ht,1,2852764.0,2852764.0,6.779648,0.0099899602
ui,1,5817995.1,5817995.1,13.826577,0.0002676048
Residuals,180,75741024.7,420783.5,NA,NA


## 3. F-test

분산분석표에서 age와 ptl이 유의하지 않으므로 두 변수를 제거한 모형을 만들고 두 변수의 중요도를 평가하기 위해 F-test를 실시한다. 
- **F-test** : 두 모형의 설명력을 비교, 제거된 변수들의 기여도를 평가하는 기능
- 형식 : **anova(작은 모형, 큰 모형)**

In [8]:
out2 <- lm(bwt~ lwt+factor(race)+smoke+ht+ui, data=birthwt)
anova(out2, out)

Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
182,75937505,NA,NA,NA,NA
180,75741025,2,196480.1,0.2334693,0.7920213


F-test 결과 p값이 0.792로 매우 크므로 두 변수를 제거 할 수 있다. plt과 age를 제거한 모형의 분산분석표를 보면 모든 변수가 유의하다.

In [10]:
anova(out2)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
lwt,1,3448639,3448639,8.265378,0.0045225781
factor(race),2,5076610,2538305,6.083575,0.0027701458
smoke,1,6281818,6281818,15.055680,0.0001457855
ht,1,2871867,2871867,6.883027,0.0094402265
ui,1,6353218,6353218,15.226806,0.0001340886
Residuals,182,75937505,417239,NA,NA


summary()로 최종모형을 살표보면 $R^2=0.2404$로 모형이 반응변수 24.04%를 설명한다. 설명력이 낮지만 통제 불가능한 변인들이 많은 연구에서 이 정도의 $R^2$는 결코 작다고 할 수 없다.

----